In [1]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from scipy.stats import rankdata
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import textstat
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple
import scml
import mylib

In [2]:
class Conf(NamedTuple):
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pretrained_dir: str = "pretrained/"
    dtfy_model_max_length: int = 512
    dtfy_batch_size: int = 256
    dtfy_models: Dict[str, str] = {
        "dto_": f"{pretrained_dir}unitaryai/detoxify/toxic_original-c1212f89.ckpt",
        "dtu_": f"{pretrained_dir}unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt",
        "dtm_": f"{pretrained_dir}unitaryai/detoxify/multilingual_debiased-0b549669.ckpt"
    }
    dtfy_configs: Dict[str, str] = {
        "dto_": f"{pretrained_dir}bert-base-uncased",
        "dtu_": f"{pretrained_dir}roberta-base",
        "dtm_": f"{pretrained_dir}xlm-roberta-base"
    }
    tweeteval_model_max_length: int = 512
    tweeteval_batch_size: int = 128
    tweeteval_models: Dict[str, str] = {
        "te_roberta_off": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-offensive",
        "te_roberta_emo_anger": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-emotion",
        "te_roberta_snt_neg": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-sentiment",
        "te_roberta_iro": f"{pretrained_dir}cardiffnlp/twitter-roberta-base-irony",
    }
    tweeteval_label_index: Dict[str, int] = {
        "te_roberta_off": 1,
        "te_roberta_emo_anger": 0,
        "te_roberta_snt_neg": 0,
        "te_roberta_iro": 1,
    }
    hatebert_model_max_length: int = 512
    hatebert_batch_size: int = 128
    hatebert_models: Dict[str, str] = {
        "hb_bert_off": f"{pretrained_dir}hatebert/bert-offenseval",
        "hb_bert_abu" : f"{pretrained_dir}hatebert/bert-abuseval",
        "hb_hatebert_off": f"{pretrained_dir}hatebert/hatebert-offenseval",
        "hb_hatebert_abu" : f"{pretrained_dir}hatebert/hatebert-abuseval",
    }
    em_max_seq_length: int = 128
    em_batch_size: int = 1000
    em_models: Dict[str, str] = {
        "paraphrase-MiniLM-L6-v2": f"{pretrained_dir}sentence-transformers/paraphrase-MiniLM-L6-v2"
    }
        
        
conf = Conf()
print(conf)
if conf.device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Conf(device=device(type='cuda'), pretrained_dir='pretrained/', dtfy_model_max_length=512, dtfy_batch_size=256, dtfy_models={'dto_': 'pretrained/unitaryai/detoxify/toxic_original-c1212f89.ckpt', 'dtu_': 'pretrained/unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt', 'dtm_': 'pretrained/unitaryai/detoxify/multilingual_debiased-0b549669.ckpt'}, dtfy_configs={'dto_': 'pretrained/bert-base-uncased', 'dtu_': 'pretrained/roberta-base', 'dtm_': 'pretrained/xlm-roberta-base'}, tweeteval_model_max_length=512, tweeteval_batch_size=128, tweeteval_models={'te_roberta_off': 'pretrained/cardiffnlp/twitter-roberta-base-offensive', 'te_roberta_emo_anger': 'pretrained/cardiffnlp/twitter-roberta-base-emotion', 'te_roberta_snt_neg': 'pretrained/cardiffnlp/twitter-roberta-base-sentiment', 'te_roberta_iro': 'pretrained/cardiffnlp/twitter-roberta-base-irony'}, tweeteval_label_index={'te_roberta_off': 1, 'te_roberta_emo_anger': 0, 'te_roberta_snt_neg': 0, 'te_roberta_iro': 1}, hatebert_model_max_length=512, hat

In [3]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
score_map: Dict[str, float] = {}
df = pd.read_csv("input/ruddit/Ruddit.csv", engine="c", low_memory=False)
for t in df.itertuples():
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    score_map[k] = getattr(t, "offensiveness_score")

In [5]:
%%time
df = pd.read_csv("input/ruddit/ruddit_with_text.csv", engine="c", low_memory=False)
blacklist = {"[deleted]", "[removed]"}
rows = []
for t in df.itertuples():
    text = getattr(t, "txt")
    s = text.strip().lower()
    if len(s)==0 or s in blacklist:
        continue
    k = getattr(t, "post_id") + "_" + getattr(t, "comment_id")
    rows.append({"bws": score_map[k], "text": text})
df = pd.DataFrame.from_records(rows)
df["bws"] = df["bws"].astype(np.float32) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   bws     5710 non-null   float32
 1   text    5710 non-null   object 
dtypes: float32(1), object(1)
memory usage: 67.0+ KB
Wall time: 51 ms


In [6]:
scml.find_missing_values(df)

,Total,Percent,Type
bws,0,0.0,float32
text,0,0.0,object


# Severity rating label

In [7]:
# Force unique ranks
col = "label"
df[col] = rankdata(df["bws"], method='ordinal')
df[col] = df[col].astype(np.int32)

# Preprocess Text

In [8]:
def preprocess(row) -> str:
    return mylib.preprocess(row["text"])


col = "text"
df[col] = df.progress_apply(preprocess, axis=1)

100%|█████████████████████████████████████████| 5710/5710 [00:02<00:00, 2425.37it/s]


In [9]:
cols = ["bws", "text"]
df[cols].sample(40).sort_values("bws").head(40)

,bws,text
3470,-0.667,All New Materials and Buttersnips too - man that first album had some great songs.
3264,-0.479,probably conventional RPGs are not your cup of tea then? have you tried other genres like Assassin's Creed or Mass effect or Skyrim which has RPG elements but not conventional RPG?
4842,-0.457,Exposing the number of bots would crash their stock
1568,-0.438,None of my friends throw a New Year's eve party. you are lucky!!!
2727,-0.375,It looks like a high ranking dark wraith perhaps if you steal enough humanity you get an upgrade to the dark sign? Looks cool AF. Either a mini boss or a normal enemy of the city... ;D
2851,-0.312,"Not really. California came at a point when it was clear Bernie had lost, so it is not surprising that Clinton won."
1820,-0.312,I started lifting and taking care of myself late. The way women treat me changed 180. Even guys are doing me favours now.
1659,-0.298,"Short guys usually cannot guy things off the rack, either. Clothes are not made for short, thin men."
605,-0.229,"Considering how closely prisoners are monitored, I suspect the reporting rate is much higher there than in the general population."
3828,-0.188,"she wont do any time, she will be fined and move on.she HOPEFULLY will be smarter in the future"


# Character level features

In [10]:
%%time
col = "length"
df[col] = df["text"].str.len()
df[col] = df[col].astype(np.int16)
char_fs = [col]

Wall time: 4.02 ms


In [11]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row["text"])


def letter_frac(row) -> float:
    return mylib.letter_frac(row["text"])


def space_frac(row) -> float:
    return mylib.space_frac(row["text"])


def punc_frac(row) -> float:
    return mylib.punc_frac(row["text"])


def upper_frac(row) -> float:
    return mylib.upper_frac(row["text"])

In [12]:
col = "digit_frac"
df[col] = df.progress_apply(digit_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 39889.67it/s]


In [13]:
col = "letter_frac"
df[col] = df.progress_apply(letter_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 39082.62it/s]


In [14]:
col = "space_frac"
df[col] = df.progress_apply(space_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 39753.14it/s]


In [15]:
col = "punc_frac"
df[col] = df.progress_apply(punc_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 35680.52it/s]


In [16]:
col = "upper_frac"
df[col] = df.progress_apply(upper_frac, axis=1)
df[col] = df[col].astype(np.float32)
char_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 40204.06it/s]


In [17]:
print(char_fs)

['length', 'digit_frac', 'letter_frac', 'space_frac', 'punc_frac', 'upper_frac']


# Textstat features

In [18]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row["text"])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row["text"])


def sentence_count(row) -> int:
    return textstat.sentence_count(row["text"])


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row["text"])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row["text"])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row["text"])


def smog_index(row) -> float:
    return textstat.smog_index(row["text"])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row["text"])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row["text"])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row["text"])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row["text"])

In [19]:
col = "flesch_reading_ease"
df[col] = df.progress_apply(flesch_reading_ease, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs = []
textstat_fs.append(col)

100%|█████████████████████████████████████████| 5710/5710 [00:00<00:00, 6213.75it/s]


In [20]:
col = "flesch_kincaid_grade"
df[col] = df.progress_apply(flesch_kincaid_grade, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 12816.05it/s]


In [21]:
col = "syllable_count"
df[col] = df.progress_apply(syllable_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 19755.79it/s]


In [22]:
col = "lexicon_count"
df[col] = df.progress_apply(lexicon_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 67155.34it/s]


In [23]:
col = "sentence_count"
df[col] = df.progress_apply(sentence_count, axis=1)
df[col] = df[col].astype(np.int16)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 40784.23it/s]


In [24]:
col = "gunning_fog"
df[col] = df.progress_apply(gunning_fog, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 10415.25it/s]


In [25]:
col = "smog_index"
df[col] = df.progress_apply(smog_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 16205.41it/s]


In [26]:
col = "automated_readability_index"
df[col] = df.progress_apply(automated_readability_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 27844.86it/s]


In [27]:
col = "coleman_liau_index"
df[col] = df.progress_apply(coleman_liau_index, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 23738.04it/s]


In [28]:
col = "linsear_write_formula"
df[col] = df.progress_apply(linsear_write_formula, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 11700.77it/s]


In [29]:
col = "dale_chall_readability_score"
df[col] = df.progress_apply(dale_chall_readability_score, axis=1)
df[col] = df[col].astype(np.float32)
textstat_fs.append(col)

100%|████████████████████████████████████████| 5710/5710 [00:00<00:00, 10215.70it/s]


In [30]:
print(textstat_fs)

['flesch_reading_ease', 'flesch_kincaid_grade', 'syllable_count', 'lexicon_count', 'sentence_count', 'gunning_fog', 'smog_index', 'automated_readability_index', 'coleman_liau_index', 'linsear_write_formula', 'dale_chall_readability_score']


# TweetEval labels

In [31]:
sentences = list(df["text"])
# all TweetEval models use the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    conf.tweeteval_models["te_roberta_off"], 
    model_max_length=conf.tweeteval_model_max_length
)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/cardiffnlp/twitter-roberta-base-offensive', vocab_size=50265, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)})
model_input_names=['input_ids', 'attention_mask']


In [32]:
%%time
x = tokenizer(sentences, truncation=True, padding="max_length")
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")

dict_keys(['input_ids', 'attention_mask'])
len=5710
Wall time: 983 ms


In [34]:
batches = torch.utils.data.DataLoader(mylib.Dataset(x), batch_size=conf.tweeteval_batch_size, shuffle=False)
for col, model_dir in conf.tweeteval_models.items():    
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    model.to(conf.device)
    logits = None
    with torch.no_grad():
        for batch in tqdm(batches):
            for k, v in batch.items():
                batch[k] = v.to(conf.device)
            outputs = model(**batch)
            tmp = outputs.logits.detach().cpu()
            if logits is None:
                logits = tmp
            else:
                logits = torch.cat((logits, tmp), 0)
    logits = torch.nn.functional.softmax(logits, dim=1)
    print(f"{col} {logits.size()}\nlogits[:10]={logits[:10]}")
    df[col] = logits[:,conf.tweeteval_label_index[col]]
    df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████████████| 45/45 [04:08<00:00,  5.52s/it]


te_roberta_off=torch.Size([5710, 2])
logits[:10]=tensor([[0.9025, 0.0975],
        [0.5804, 0.4196],
        [0.6679, 0.3321],
        [0.8045, 0.1955],
        [0.7823, 0.2177],
        [0.7749, 0.2251],
        [0.8272, 0.1728],
        [0.6837, 0.3163],
        [0.7598, 0.2402],
        [0.9035, 0.0965]])


100%|███████████████████████████████████████████████| 45/45 [04:08<00:00,  5.52s/it]


te_roberta_emo_anger=torch.Size([5710, 4])
logits[:10]=tensor([[0.0376, 0.0565, 0.8628, 0.0431],
        [0.1369, 0.1012, 0.2558, 0.5061],
        [0.8166, 0.0105, 0.0391, 0.1338],
        [0.9419, 0.0041, 0.0231, 0.0308],
        [0.6140, 0.0225, 0.1561, 0.2074],
        [0.8744, 0.0085, 0.0585, 0.0586],
        [0.8556, 0.0071, 0.0948, 0.0425],
        [0.8475, 0.0155, 0.0260, 0.1111],
        [0.8054, 0.0086, 0.0704, 0.1156],
        [0.1417, 0.3192, 0.3703, 0.1687]])


100%|███████████████████████████████████████████████| 45/45 [04:08<00:00,  5.52s/it]


te_roberta_snt_neg=torch.Size([5710, 3])
logits[:10]=tensor([[0.0407, 0.7561, 0.2032],
        [0.4268, 0.5488, 0.0244],
        [0.7662, 0.2259, 0.0080],
        [0.6066, 0.3729, 0.0206],
        [0.5169, 0.4232, 0.0599],
        [0.5795, 0.3848, 0.0357],
        [0.3226, 0.6394, 0.0380],
        [0.7787, 0.2147, 0.0066],
        [0.7998, 0.1863, 0.0139],
        [0.1055, 0.6091, 0.2854]])


100%|███████████████████████████████████████████████| 45/45 [04:08<00:00,  5.52s/it]

te_roberta_iro=torch.Size([5710, 2])
logits[:10]=tensor([[0.7999, 0.2001],
        [0.4219, 0.5781],
        [0.3392, 0.6608],
        [0.8240, 0.1760],
        [0.8869, 0.1131],
        [0.4666, 0.5334],
        [0.7453, 0.2547],
        [0.0874, 0.9126],
        [0.8681, 0.1319],
        [0.6555, 0.3445]])


# HateBert labels

In [35]:
# all Hatebert models use the same tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    conf.hatebert_models["hb_hatebert_off"], 
    model_max_length=conf.hatebert_model_max_length
)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/hatebert/hatebert-offenseval', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']


In [36]:
%%time
x = tokenizer(sentences, truncation=True, padding="max_length")
print(f"{repr(x.keys())}\nlen={len(x['input_ids'])}")

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
len=5710
Wall time: 1.01 s


In [37]:
batches = torch.utils.data.DataLoader(mylib.Dataset(x), batch_size=conf.hatebert_batch_size, shuffle=False)
for col, model_dir in conf.hatebert_models.items():    
    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()
    model.to(conf.device)
    logits = None
    with torch.no_grad():
        for batch in tqdm(batches):
            for k, v in batch.items():
                batch[k] = v.to(conf.device)
            outputs = model(**batch)
            tmp = outputs.logits.detach().cpu()
            if logits is None:
                logits = tmp
            else:
                logits = torch.cat((logits, tmp), 0)
    logits = torch.nn.functional.softmax(logits, dim=1)
    print(f"{col} {logits.size()}\nlogits[:10]={logits[:10]}")
    df[col] = logits[:,1]
    df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████████████| 45/45 [04:13<00:00,  5.62s/it]


hb_bert_off=torch.Size([5710, 2])
logits[:10]=tensor([[0.9598, 0.0402],
        [0.7718, 0.2282],
        [0.7567, 0.2433],
        [0.9064, 0.0936],
        [0.8356, 0.1644],
        [0.7731, 0.2269],
        [0.5952, 0.4048],
        [0.8502, 0.1498],
        [0.7770, 0.2230],
        [0.9258, 0.0742]])


100%|███████████████████████████████████████████████| 45/45 [04:13<00:00,  5.62s/it]


hb_bert_abu=torch.Size([5710, 2])
logits[:10]=tensor([[0.9910, 0.0090],
        [0.9717, 0.0283],
        [0.9545, 0.0455],
        [0.9631, 0.0369],
        [0.9727, 0.0273],
        [0.8744, 0.1256],
        [0.9001, 0.0999],
        [0.8544, 0.1456],
        [0.8708, 0.1292],
        [0.9881, 0.0119]])


100%|███████████████████████████████████████████████| 45/45 [04:15<00:00,  5.68s/it]


hb_hatebert_off=torch.Size([5710, 2])
logits[:10]=tensor([[0.9377, 0.0623],
        [0.5526, 0.4474],
        [0.7553, 0.2447],
        [0.7828, 0.2172],
        [0.8740, 0.1260],
        [0.9279, 0.0721],
        [0.8304, 0.1696],
        [0.7886, 0.2114],
        [0.8038, 0.1962],
        [0.8965, 0.1035]])


100%|███████████████████████████████████████████████| 45/45 [04:12<00:00,  5.62s/it]

hb_hatebert_abu=torch.Size([5710, 2])
logits[:10]=tensor([[0.9846, 0.0154],
        [0.9717, 0.0283],
        [0.9570, 0.0430],
        [0.9699, 0.0301],
        [0.9700, 0.0300],
        [0.9775, 0.0225],
        [0.9800, 0.0200],
        [0.9733, 0.0267],
        [0.9407, 0.0593],
        [0.9795, 0.0205]])


# Detoxify labels

In [38]:
dtfy_fs = []
for prefix, checkpoint in tqdm(conf.dtfy_models.items()):
    res = mylib.detoxify_labels(
        sentences,
        checkpoint=checkpoint,
        config_dir=conf.dtfy_configs[prefix],
        model_max_length=conf.dtfy_model_max_length,
        device=conf.device,
        batch_size=conf.dtfy_batch_size
    )
    for k, v in res.items():
        col = prefix + k
        df[col] = v
        df[col] = df[col].astype(np.float32)
        dtfy_fs.append(col)
    gc.collect()

100%|█████████████████████████████████████████████████| 3/3 [03:58<00:00, 79.63s/it]


In [39]:
print(dtfy_fs)

['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']


# Embeddings

In [40]:
model = SentenceTransformer(conf.em_models["paraphrase-MiniLM-L6-v2"], device=conf.device)
model.max_seq_length = conf.em_max_seq_length
em = model.encode(sentences=sentences, batch_size=conf.em_batch_size, show_progress_bar=True, convert_to_numpy=True)
print(f"em.shape={em.shape}")

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

em.shape=(5710, 384)


In [41]:
%%time
em_size = em.shape[1]
em_cols = [f"zz{i:04d}" for i in range(em_size)]
df[em_cols] = em
df[em_cols] = df[em_cols].astype(np.float32)
del sentences

s:\dev\seahrh\kaggle-jigsaw-toxic-severity-rating\env\lib\site-packages\pandas\core\frame.py:3673: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Wall time: 245 ms


# Review data

In [42]:
col = "worker"
df[col] = 0
df[col] = df[col].astype(np.int8)
cols = ["label", "bws", "worker"]
cols += char_fs + textstat_fs + dtfy_fs 
cols += list(conf.hatebert_models.keys()) 
cols += list(conf.tweeteval_models.keys()) 
df[cols].describe(percentiles=percentiles)

,label,bws,worker,length,digit_frac,letter_frac,space_frac,punc_frac,upper_frac,flesch_reading_ease,flesch_kincaid_grade,syllable_count,lexicon_count,sentence_count,gunning_fog,smog_index,automated_readability_index,coleman_liau_index,linsear_write_formula,dale_chall_readability_score,dto_toxicity,dto_severe_toxicity,dto_obscene,dto_threat,dto_insult,dto_identity_attack,dtu_toxicity,dtu_severe_toxicity,dtu_obscene,dtu_identity_attack,dtu_insult,dtu_threat,dtu_sexual_explicit,dtm_toxicity,dtm_severe_toxicity,dtm_obscene,dtm_identity_attack,dtm_insult,dtm_threat,dtm_sexual_explicit,hb_bert_off,hb_bert_abu,hb_hatebert_off,hb_hatebert_abu,te_roberta_off,te_roberta_emo_anger,te_roberta_snt_neg,te_roberta_iro
count,5710.00000,5710.000000,5710.0,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5.710000e+03,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000,5710.000000
mean,2855.50000,-0.027706,0.0,197.564098,0.003542,0.788830,0.177722,0.029905,0.030517,75.253105,6.685254,50.067426,36.371278,2.319790,8.998004,3.062995,7.482505,6.631683,8.562484,8.299163,0.177856,0.013080,0.113673,0.008411,0.059683,0.011454,0.195873,4.698273e-03,0.113027,0.015234,0.081492,0.011010,0.040167,0.203075,0.005742,0.105158,0.013474,0.083563,0.014735,0.044618,0.332577,0.166173,0.329977,0.144314,0.350061,0.507189,0.508503,0.313383
std,1648.47935,0.334195,0.0,172.016744,0.012983,0.034318,0.021473,0.024070,0.049035,19.903004,4.430519,44.035705,31.194250,1.722369,4.645890,4.695458,5.593331,3.883116,5.906460,2.270676,0.326334,0.052992,0.282462,0.056317,0.175381,0.064708,0.341383,2.272458e-02,0.285412,0.074354,0.207780,0.069685,0.149433,0.345334,0.027226,0.272925,0.075226,0.214901,0.078828,0.155314,0.332031,0.278667,0.284761,0.221301,0.251317,0.357695,0.319341,0.276529
min,1.00000,-0.889000,0.0,15.000000,0.000000,0.400000,0.040541,0.000000,0.000000,-48.980000,-2.500000,3.000000,2.000000,1.000000,0.800000,0.000000,-8.700000,-10.160000,0.000000,0.200000,0.000506,0.000080,0.000141,0.000086,0.000164,0.000121,0.000286,9.529070e-07,0.000017,0.000052,0.000061,0.000012,0.000009,0.000178,0.000009,0.000063,0.000045,0.000095,0.000014,0.000012,0.008860,0.002739,0.007778,0.007624,0.021641,0.004902,0.000799,0.014366
1%,58.09000,-0.667000,0.0,24.000000,0.000000,0.672759,0.117647,0.000000,0.000000,19.430301,-1.500000,6.000000,5.000000,1.000000,2.000000,0.000000,-3.000000,-3.003700,1.500000,0.350000,0.000556,0.000087,0.000158,0.000096,0.000171,0.000134,0.000360,1.087282e-06,0.000021,0.000063,0.000092,0.000015,0.000011,0.000281,0.000013,0.000095,0.000066,0.000149,0.000021,0.000017,0.013618,0.003827,0.021895,0.010283,0.049634,0.010852,0.001768,0.022520
5%,286.45000,-0.521000,0.0,33.000000,0.000000,0.733333,0.141414,0.000000,0.005062,41.869999,0.500000,8.000000,6.000000,1.000000,2.400000,0.000000,-0.200000,0.320000,2.000000,5.620000,0.000614,0.000093,0.000165,0.000103,0.000175,0.000138,0.000410,1.200621e-06,0.000024,0.000070,0.000103,0.000017,0.000012,0.000357,0.000016,0.000115,0.000078,0.000177,0.000027,0.000020,0.021350,0.005177,0.036465,0.013695,0.074244,0.024071,0.008642,0.034839
10%,571.90000,-0.426000,0.0,42.000000,0.000000,0.750000,0.151515,0.010417,0.008333,50.669998,1.700000,10.000000,8.000000,1.000000,3.200000,0.000000,1.290000,1.820000,2.741667,6.240000,0.000668,0.000097,0.000170,0.000108,0.000177,0.000140,0.000454,1.271093e-06,0.000026,0.000076,0.000111,0.000019,0.000013,0.000417,0.000019,0.000129,0.000086,0.000200,0.000031,0.000022,0.029108,0.006430,0.049953,0.016057,0.091896,0.039984,0.032663,0.046494
20%,1142.80000,-0.312000,0.0,60.000000,0.000000,0.769841,0.1619

In [43]:
cols += em_cols
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5710 entries, 0 to 5709
Data columns (total 432 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    label                         5710 non-null   int32  
 1    bws                           5710 non-null   float32
 2    worker                        5710 non-null   int8   
 3    length                        5710 non-null   int16  
 4    digit_frac                    5710 non-null   float32
 5    letter_frac                   5710 non-null   float32
 6    space_frac                    5710 non-null   float32
 7    punc_frac                     5710 non-null   float32
 8    upper_frac                    5710 non-null   float32
 9    flesch_reading_ease           5710 non-null   float32
 10   flesch_kincaid_grade          5710 non-null   float32
 11   syllable_count                5710 non-null   int16  
 12   lexicon_count                 5710 non-null   

In [44]:
%%time
df[cols].to_parquet("output/tra.parquet", index=False)

Wall time: 229 ms
